In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# 读入评分数据

In [2]:
#读取数据
dtype={"msno": object,"song_id": object,"source_system_tab": object,"source_screen_name": object,"source_type": object,"target": int}
dpath = '../Data/'
df_kkbox = pd.read_csv(dpath+'train.csv', dtype=dtype)
df_kkbox.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [3]:
df_kkbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 6 columns):
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                int64
dtypes: int64(1), object(5)
memory usage: 337.7+ MB


7377418个样本，5个特征1个标签列

# 查看缺省值情况

In [4]:
for key in dtype:
    print(key,'缺省值数：',len(df_kkbox[pd.isna(df_kkbox[key])==True]))

msno 缺省值数： 0
song_id 缺省值数： 0
source_system_tab 缺省值数： 24849
source_screen_name 缺省值数： 414804
source_type 缺省值数： 21539
target 缺省值数： 0


source_system_tab，source_screen_name和source_type各有24849,414804和21539个缺省值

# 计算用户数、音乐数目

In [5]:
n_users = df_kkbox['msno'].unique().shape[0]
n_items = df_kkbox['song_id'].unique().shape[0]
print ('Number of users = ' + str(n_users) + '\n'+ 'Number of items = ' + str(n_items) )

Number of users = 30755
Number of items = 359966


有继续订阅的用户数为30755，少于被继续订阅的物品数量359966

In [6]:
df_clicked = df_kkbox[df_kkbox['target']==1]
user_freq = df_clicked['msno'].value_counts() 
item_freq = df_clicked['song_id'].value_counts()
del df_clicked

In [8]:
user_freq

o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=    3288
KGXNZ/H3VxvET/+rGxlrAe7Gpz2eKMXyuSg3xh8Ij1M=    2818
FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=    2791
MXIMDXO0j3UpaT7FvOSGW6Y5zfhlh+xYjTqGoUdMzEE=    2069
dU4RbzpIRRd/EkA9Xncpy9CglzDBZp7nKMfdnfr3Aj8=    1978
4DlS5LiANEKlMTQD08dakyV1JWwPYI/n1wyjx4ImjPc=    1814
JkQacE3rvmhh65R04eDLbu+M7MCkpzmHwMGrbZo0puc=    1806
uCDJmSDTzA5PpnYGqKvNmFbwBIoS3jIXFbO5YfoCv/Y=    1794
1wiu2UfN0BNhHfPTrDS3S/rQkX/vnwDoKH6ODTUdH9k=    1737
cqjRBV/jWN2ujhc+z/4tz+Mj6xEfflAAt6qBXCqxKvw=    1735
7xiBI5xU3P2/IiR9teB7jySfzWo96JGikGajteLd3Cs=    1715
HVcWdf8CEo9s6qwt5V7TpoPYJfRlQSTA6b7kxEAImpc=    1710
mDJCU+fKu/mbdk9l4SmPYiJwpeMyK44o9wmG1X3735A=    1607
V39VxDdE9SgMQZnX58oQChm9W056pw2lC3TiifMfkKw=    1602
LThaiVqGGnVTPmTcmwN/LLo4fVb5dzkduzd7s1SgzIA=    1578
EZmIP7zKgqKwpEHnuDjRIkzCmcLNlHn4hkiglZ5LxK8=    1539
00qmlWC3H/cjf2Ig1li5xIO0VNDRgBX0f9ilouDBEZs=    1533
x+k/zTEqdgYq6O+r1iDjKRaQnaCS5118gzBe8c12r1Y=    1503
K7obY5lUlZTd6a/byE9NG/e7Q/dlgHfppLbYk8OL7os=  

最活跃的用户，连续订阅了3288次，最不活跃的用户才1次（可能为游客，或者误点击，或纯粹不喜欢不愿继续订阅），用户活跃程度分化严重

In [9]:
item_freq

reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=    10885
T86YHdD4C9JSc274b1IlMkLuNdz4BQRB50fWWE7hx9g=    10556
FynUyq0+drmIARmK1JZ/qcjNZ7DKkqTY6/0O0lTzNUI=     9808
wBTWuHbjdjxnG1lQcbqnK4FddV24rUhuyrYLd9c/hmk=     9411
PgRtmmESVNtWjoZHO5a1r21vIz9sVZmcJJpFCbRa1LI=     9004
U9kojfZSKaiWOW94PKh1Riyv/zUWxmBRmv0XInQWLGw=     8787
YN4T/yvvXtYrBVN8KTnieiQohHL3T9fnzUkbLWcgLro=     8780
M9rAajz4dYuRhZ7jLvf9RRayVA3os61X/XXHEuW4giA=     8403
43Qm2YzsP99P5wm37B1JIhezUcQ/1CDjYlQx6rBbz2U=     8112
J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=     7903
cy10N2j2sdY/X4BDUcMu2Iumfz7pV3tqE5iEaup2yGI=     7725
750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=     7608
IKMFuL0f5Y8c63Hg9BXkeNJjE0z8yf3gMt/tOxF4QNE=     7224
+SstqMwhQPBQFTPBhLKPT642IiBDXzZFwlzsLl4cGXo=     7061
DLBDZhOoW7zd7GBV99bi92ZXYUS26lzV+jJKbHshP5c=     6901
v/3onppBGoSpGsWb8iaCIO8eX5+iacbH5a4ZUhT7N54=     6879
p/yR06j/RQ2J6yGCFL0K+1R06OeG+eXcwxRgOHDo/Tk=     6536
Xpjwi8UAE2Vv9PZ6cZnhc58MCtl3cKZEO1sdAkqJ4mo=     6399
OaEbZ6TJ1NePtNUeEgWsvFLeopkS

最热门的歌曲，被继续订阅了10885次，很多歌曲则小于等于1次，长尾效应很明显。

In [10]:
df_kkbox['source_system_tab'].unique()

array(['explore', 'my library', 'search', 'discover', nan, 'radio',
       'listen with', 'notification', 'settings'], dtype=object)

In [18]:
for source_system_tab in ['explore', 'my library', 'search', 'discover', 'radio','listen with', 'notification', 'settings']:
    data = df_kkbox[df_kkbox['source_system_tab']==source_system_tab]
    ratio = len(data[data['target']==1])*100.0/len(data)
    print(source_system_tab,'被继续订阅的概率：',ratio)
    
data = df_kkbox[pd.isna(df_kkbox['source_system_tab'])==True]
print('nan 被继续订阅的概率：',len(data[data['target']==1])*100.0/len(data))

explore 被继续订阅的概率： 42.21460086097565
my library 被继续订阅的概率： 61.965897094224
search 被继续订阅的概率： 42.136194299246256
discover 被继续订阅的概率： 41.57697228223262
radio 被继续订阅的概率： 22.26615845152412
listen with 被继续订阅的概率： 32.65807995628127
notification 被继续订阅的概率： 37.80113177041229
settings 被继续订阅的概率： 59.09090909090909
nan 被继续订阅的概率： 51.036259004386494


source_system_tab: 触发事件的类型/tab，用于表示app的功能类型，与是否继续订阅有联系，但是无法作为R矩阵的评分

In [17]:
df_kkbox['source_screen_name'].unique()

array(['Explore', 'Local playlist more', nan, 'My library',
       'Online playlist more', 'Album more', 'Discover Feature',
       'Unknown', 'Discover Chart', 'Radio', 'Artist more', 'Search',
       'Others profile more', 'Search Trends', 'Discover Genre',
       'My library_Search', 'Search Home', 'Discover New',
       'Self profile more', 'Concert', 'Payment'], dtype=object)

In [19]:
source_screen_name=['Explore', 'Local playlist more', 'My library','Online playlist more', 'Album more', 'Discover Feature',
       'Unknown', 'Discover Chart', 'Radio', 'Artist more', 'Search', 'Others profile more', 'Search Trends', 'Discover Genre',
       'My library_Search', 'Search Home', 'Discover New','Self profile more', 'Concert', 'Payment']

for screen_name in source_screen_name:
    data = df_kkbox[df_kkbox['source_screen_name']==screen_name]
    ratio = len(data[data['target']==1])*100.0/len(data)
    print(screen_name,'被继续订阅的概率：',ratio)
    
data = df_kkbox[pd.isna(df_kkbox['source_screen_name'])==True]
print('nan 被继续订阅的概率：',len(data[data['target']==1])*100.0/len(data))

Explore 被继续订阅的概率： 44.81490696967184
Local playlist more 被继续订阅的概率： 63.69833734072403
My library 被继续订阅的概率： 65.71729402474335
Online playlist more 被继续订阅的概率： 41.4939031690236
Album more 被继续订阅的概率： 39.055255666942756
Discover Feature 被继续订阅的概率： 36.41042227917755
Unknown 被继续订阅的概率： 33.915451356839576
Discover Chart 被继续订阅的概率： 51.70318920892267
Radio 被继续订阅的概率： 21.72564161469607
Artist more 被继续订阅的概率： 41.68578095226777
Search 被继续订阅的概率： 47.17491884068653
Others profile more 被继续订阅的概率： 31.262915334869547
Search Trends 被继续订阅的概率： 37.69806338028169
Discover Genre 被继续订阅的概率： 34.788691272718424
My library_Search 被继续订阅的概率： 61.122306619128814
Search Home 被继续订阅的概率： 35.3582554517134
Discover New 被继续订阅的概率： 45.5531181447822
Self profile more 被继续订阅的概率： 42.45283018867924
Concert 被继续订阅的概率： 51.06382978723404
Payment 被继续订阅的概率： 66.66666666666667
nan 被继续订阅的概率： 46.97881409051022


source_screen_name: 用户看到的布局的名字，与是否继续订阅有联系，但是无法作为R矩阵的评分标准

In [20]:
df_kkbox['source_type'].unique()

array(['online-playlist', 'local-playlist', 'local-library',
       'top-hits-for-artist', 'album', nan, 'song-based-playlist',
       'radio', 'song', 'listen-with', 'artist', 'topic-article-playlist',
       'my-daily-playlist'], dtype=object)

In [21]:
source_type=['online-playlist', 'local-playlist', 'local-library', 'top-hits-for-artist', 'album', 'song-based-playlist',
       'radio', 'song', 'listen-with', 'artist', 'topic-article-playlist','my-daily-playlist']

for src_type in source_type:
    data = df_kkbox[df_kkbox['source_type']==src_type]
    ratio = len(data[data['target']==1])*100.0/len(data)
    print(src_type,'被继续订阅的概率：',ratio)
    
data = df_kkbox[pd.isna(df_kkbox['source_type'])==True]
print('nan 被继续订阅的概率：',len(data[data['target']==1])*100.0/len(data))

online-playlist 被继续订阅的概率： 42.49854161034674
local-playlist 被继续订阅的概率： 65.77193393626511
local-library 被继续订阅的概率： 63.20980950287853
top-hits-for-artist 被继续订阅的概率： 41.85366867006284
album 被继续订阅的概率： 39.34206777502179
song-based-playlist 被继续订阅的概率： 38.04595135065811
radio 被继续订阅的概率： 21.973509083871342
song 被继续订阅的概率： 43.74596480904864
listen-with 被继续订阅的概率： 31.965028365190157
artist 被继续订阅的概率： 57.274522712310734
topic-article-playlist 被继续订阅的概率： 49.42826514204038
my-daily-playlist 被继续订阅的概率： 37.55656108597285
nan 被继续订阅的概率： 52.52332977389851


source_type: 用户在app上播放音乐的入口的类型，与是否继续订阅有联系，但显然无法作为R矩阵的评分标准

In [22]:
df_kkbox['target'].unique()

array([1, 0])

In [32]:
t = len(df_kkbox[df_kkbox['target']==1])
f = len(df_kkbox[df_kkbox['target']==0])
print('正样本数：',t)
print('负样本数：',t)
print('正负比例：',1.0*t/f)

正样本数： 3714656
负样本数： 3714656
正负比例： 1.0141679967194155


正负样本基本平衡。可作为用户对item的喜爱程度，作为R矩阵的评分标准